## なぜ営業キャッシュフロー・マージンが高いほうが良い
そのような<font color="red">**高利益体質**</font>の企業は現在のような厳しい経営環境でも企業の「稼ぐ力」が低下しにくいから
## 見つけ方    
    
+ 【ルールその１】営業キャッシュフローは年々着実に増えていることが望ましい
+ 【ルールその２】営業キャッシュフローはその年の利益より大きくなければいけない
+ 【ルールその３】営業キャッシュフロー・マージンは15％以上の会社を狙え

#### 営業キャッシュフローとは、
ある企業が商品やサービスを売ることで得たお金、つまり売上高から、原材料費などの支出を引き算した「現金収支」を指します。



## 【ルールその１】営業キャッシュフローは年々着実に増えていることが望ましい

営業キャッシュフロー（CFPS）を経年で比較してください。一昨年より去年、去年より今年のCFPSの数字が大きければ合格です。エヌビディアの場合2020年度（＝カレンダー・イヤーでは2019年）は需要サイクルの暗転という事情があり、かなり苦しい年でした。それでもなんとか前年並みのCFPSを達成しています。

Quantopian では、`Gross Income Margin` を使うのがいいと思われる。比較しやすい。

+ [gross_mgn_af, gross_mgn_ltm, gross_mgn_qf, gross_mgn_saf](https://www.quantopian.com/docs/data-reference/factset_fundamentals#gross-mgn-af-gross-mgn-ltm-gross-mgn-qf-gross-mgn-saf)
    + Gross Income Margin
    + Represents gross income margin. This is calculated as Gross Income (gross_inc) divided by Net Sales or Revenue (sales), multiplied by 100.
    + Note: If the denominator is negative, the formula will return NA.

PerShare で見たい場合は、これでもよいかな。

+ [Quantopian Cash Flow Per Share - 1 Year Growth](https://www.quantopian.com/docs/data-reference/factset_fundamentals#cf-ps-gr-af-cf-ps-gr-qf-cf-ps-gr-saf)
    + `cf_ps_gr_af`, `cf_ps_gr_qf`, `cf_ps_gr_saf`
    + Represents the one-year growth rate of cash flow per share. This is calculated by taking the year over year percent change of Net Cash Flow from Operations Per Share. Note: If the growth goes from negative to positive, the formula will return NA.



In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.factset import Fundamentals
from quantopian.pipeline.data.morningstar import Fundamentals as morningstar
import quantopian.pipeline.data.factset.estimates as fe

from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets


# Construct a factor that simply gets the most recent quarterly sales value.
quarterly_sales = Fundamentals.sales_qf.latest
cfps = Fundamentals.cf_ps_gr_af.latest
gross_mgn_af = Fundamentals.gross_mgn_af.latest

fq0_cfps_act = fe.Actuals.slice('CFPS', 'af', 0)
fq0_cfps_act_value = fq0_cfps_act.actual_value

fq0_eps_act = fe.Actuals.slice('EPS', 'af', 0)
fq0_eps_act_value = fq0_eps_act.actual_value



morningstar_industry_code = morningstar.morningstar_industry_code.latest
morningstar_industry_group_code = morningstar.morningstar_industry_group_code.latest


# Add the factor to the pipeline.
pipe = Pipeline(
    columns={
#         'sales': quarterly_sales,
#         'cfps':cfps, 
        'gross_mgn_af':gross_mgn_af,
        'fq0_eps_act_value':fq0_eps_act_value.latest,
        'fq0_cfps_act_value':fq0_cfps_act_value.latest,
        
        'morningstar_industry_code':morningstar_industry_code, 
        'morningstar_industry_group_code':morningstar_industry_group_code,
    },
    domain=US_EQUITIES,
)

# Run the pipeline over a year and print the result.
df = run_pipeline(pipe, '2015-05-05', '2019-05-05')
df.head()


In [ ]:
df.xs(symbols('NVDA'), level=1)["fq0_cfps_act_value"].plot()

In [ ]:
df.xs(symbols('NVDA'), level=1)["gross_mgn_af"].describe()

In [ ]:
## 毎年 cash flow margin が上がっている銘柄を探す。
import numpy as np
df_T = df["gross_mgn_af"].unstack().dropna(axis=1)
change = df_T.pct_change().dropna() 
print(len(change.columns))
candidates = change.where(change >= 0).dropna(axis=1).columns
print(len(candidates))
candidates_desc = df_T[candidates.tolist()].dropna(axis=1).describe()


In [ ]:
candidates_desc.T[candidates_desc.T["min"] > 50].sort_values(by="std",ascending=False).head(20)

In [ ]:
x = candidates_desc.T[(candidates_desc.T["min"] > 50) & (candidates_desc.T["std"] > 1.5) ].sort_values(by="std",ascending=False).index.tolist()
df_T[x]


import matplotlib.pyplot as plt 

fig = plt.figure(figsize=(20,10))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)
num = len(df_T[x].columns)
xs = 5
ys = int(num/5)+1
for i, c in enumerate(df_T[x].columns):
    x = i % xs
    y = int(i/xs)+ys
    ax = fig.add_subplot(xs, ys, i+1)
    plt.ylim(50,100)
    ax.set_title(c.symbol)
    ax.plot(df_T[c])
    
    

In [ ]:
x = candidates_desc.T[(candidates_desc.T["min"] > 50) & (candidates_desc.T["std"] > 1.5) ].sort_values(by="std",ascending=False).index.tolist()
n = len(x)
yoko = 5
tate = int(n / yoko) + 1
print(tate,yoko)
df_T[x].plot(subplots=True, layout=(tate, yoko), figsize=(20,20), ylim=(50, 100))

## 【ルールその２】営業キャッシュフローはその年の一株当たり利益（EPS）より必ず大きくなればいけない

いまCFPS(灰色)とEPS（橙色）を比べてください。灰色の方が橙色より大きいことが望ましいです。これが逆転（＝すなわち橙色の方が大きくなってしまっている）すると粉飾決算の恐れがあります。


In [ ]:
cfps = df["fq0_cfps_act_value"].unstack().dropna(axis=1)
cfps_desc = cfps.describe().T

In [ ]:
cfps[list(set(cfps.columns.tolist()) & set(x))]


In [ ]:
mylist = list(set(cfps.columns.tolist()) & set(x))
yoko = 5
tate = int(len(mylist)/yoko)+1

cfps[mylist].plot(subplots=True, layout=(tate, yoko), figsize=(20,20))

In [ ]:
df.xs(symbols('HUBS'), level=1)[["gross_mgn_af", "fq0_cfps_act_value"]].plot(secondary_y="fq0_cfps_act_value")

In [ ]:
(cfps[mylist] > df["fq0_eps_act_value"].unstack()[mylist]).sum()

## 【ルールその３】営業キャッシュフロー・マージンは15％以上の会社を狙え!

営業キャッシュフロー・マージンとは一株当たり営業キャッシュフロー（CFPS：灰色）を一株当たり売上高（SPS：黄色）で割り算し、パーセントにした数値です。


In [ ]:
(df.xs(symbols('HUBS'), level=1)["fq0_cfps_act_value"] > df.xs(symbols('HUBS'), level=1)["fq0_eps_act_value"]).unique()